In [1]:
pip install numpy


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -e ./electre_library

Obtaining file:///Users/mrezkys/Kampus/SPK/electre_library
  Preparing metadata (setup.py) ... done
  Attempting uninstall: electre_dss
    Found existing installation: electre_dss 0.1.0
    Uninstalling electre_dss-0.1.0:
      Successfully uninstalled electre_dss-0.1.0
  DEPRECATION: Legacy editable install of electre_dss==0.1.0 from file:///Users/mrezkys/Kampus/SPK/electre_library (setup.py develop) is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for electre_dss

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m p

In [4]:
import numpy as np
import pandas as pd # Optional: For nice table display
from electre_dss import ElectreSolver

In [5]:
# 1. Define the Data

decision_matrix = np.array([
    [80, 70, 80, 70, 90], # HP1
    [80, 80, 70, 70, 90], # HP2
    [90, 70, 80, 70, 80]  # HP3
])

weights = np.array([5, 4, 3, 4, 2])

# C1 (Harga), C4 (Berat) = cost
# C2 (Kamera), C3 (Memori), C5 (Keunikan) = benefit
criteria_types = ['cost', 'benefit', 'benefit', 'cost', 'benefit']

alternative_names = ['HP1', 'HP2', 'HP3']
criteria_names = ['C1(Harga)', 'C2(Kamera)', 'C3(Memori)', 'C4(Berat)', 'C5(Keunikan)']

print("--- Input Data ---")
print("Decision Matrix:")
display(pd.DataFrame(decision_matrix, index=alternative_names, columns=criteria_names))
print(f"\nWeights: {weights}")
print(f"Criteria Types: {criteria_types}")

--- Input Data ---
Decision Matrix:


,C1(Harga),C2(Kamera),C3(Memori),C4(Berat),C5(Keunikan)
HP1,80,70,80,70,90
HP2,80,80,70,70,90
HP3,90,70,80,70,80



Weights: [5 4 3 4 2]
Criteria Types: ['cost', 'benefit', 'benefit', 'cost', 'benefit']


In [6]:
# 2. Initialize and run the solver
print("\n--- Running Solver ---")
solver = ElectreSolver(
    decision_matrix=decision_matrix,
    weights=weights,
    criteria_types=criteria_types,
    alternative_names=alternative_names,
    criteria_names=criteria_names
)

solver.solve() # This performs all calculations


--- Running Solver ---


In [7]:
# 3. Get and print the final ranking
ranking = solver.get_ranking()
print("\n------ FINAL RANKING ------")
for i, name in enumerate(ranking):
    print(f"{i+1}. {name}")


------ FINAL RANKING ------
1. HP2
2. HP1
3. HP3


In [8]:
# 4. Display detailed intermediate results for verification
print("\n------ DETAILED STEP-BY-STEP RESULTS ------")
results = solver.get_intermediate_results()
alternative_names = solver.alternative_names # Get names from solver
criteria_names = solver.criteria_names     # Get names from solver

# Step 1: Normalization (Euclidean)
print("\nStep 1: Normalized Decision Matrix (R)")
display(pd.DataFrame(results['raw_normalized_matrix'], index=alternative_names, columns=criteria_names).round(3))

# Step 2: Weighted Normalization
print("\nStep 2: Weighted Normalized Matrix (V)")
display(pd.DataFrame(results['weighted_matrix'], index=alternative_names, columns=criteria_names).round(3))

# Step 3 & 4: Concordance/Discordance Sets & Indices
print("\nSteps 3 & 4: Concordance and Discordance Analysis")
# Ensure global max diff is available and formatted correctly
global_max_diff_val = results['discordance_global_max_diff']
print(f"Global Max Difference for Discordance Denominator: {global_max_diff_val:.3f}")

for k in range(solver.num_alternatives):
    for l in range(solver.num_alternatives):
        if k == l: continue
        k_name = alternative_names[k]
        l_name = alternative_names[l]
        pair = (k, l) # Use integer indices for dict keys

        print(f'\n--- Pair ({k_name} vs {l_name}) ---')
        
        # Concordance
        # Use .get(pair, []) for safety if a pair somehow wasn't calculated
        conc_set = results['concordance_sets'].get(pair, []) 
        conc_index = results['concordance_matrix'][k, l]
        # Format set output nicely
        conc_set_str = ', '.join(conc_set) if conc_set else 'None' 
        print(f"  Concordance Set C({k_name},{l_name}): {{ {conc_set_str} }}")
        print(f"  Concordance Index C_{k+1},{l+1}: {conc_index:.1f}") # Match report format (1 decimal usually)

        # Discordance
        disc_set = results['discordance_sets'].get(pair, [])
        # Get max diff within the set for this pair
        disc_max_diff = results['discordance_max_diffs'].get(pair, 0.0) 
        disc_index = results['discordance_matrix'][k, l]
        disc_set_str = ', '.join(disc_set) if disc_set else 'None'
        print(f"  Discordance Set D({k_name},{l_name}): {{ {disc_set_str} }}")
        print(f"  Max Difference in D({k_name},{l_name}): {disc_max_diff:.3f}")
        # Show the calculation clearly
        print(f"  Discordance Index d_{k+1},{l+1}: ({disc_max_diff:.3f} / {global_max_diff_val:.3f}) = {disc_index:.3f}")

# Step 5: Thresholds
print("\nStep 5: Threshold Calculation")
c_threshold = results['concordance_threshold']
d_threshold = results['discordance_threshold']
print(f"  Concordance Threshold (c) = Sum(C_kl) / (m*(m-1)) = {np.sum(results['concordance_matrix']):.1f} / {solver.num_alternatives * (solver.num_alternatives - 1)} = {c_threshold:.3f}")
print(f"  Discordance Threshold (d) = Sum(d_kl) / (m*(m-1)) = {np.sum(results['discordance_matrix']):.3f} / {solver.num_alternatives * (solver.num_alternatives - 1)} = {d_threshold:.4f}")

# Step 6: Dominance Matrices
print("\nStep 6: Dominance Matrices")
print(f"  Concordance Dominance Matrix (F): C_kl >= {c_threshold:.3f}")
display(pd.DataFrame(results['concordance_dominance_matrix'], index=alternative_names, columns=alternative_names))
print(f"  Discordance Dominance Matrix (G): d_kl <= {d_threshold:.4f}")
display(pd.DataFrame(results['discordance_dominance_matrix'], index=alternative_names, columns=alternative_names))

# Step 7: Aggregate Dominance Matrix
print("\nStep 7: Aggregate Dominance Matrix (E) = F * G")
display(pd.DataFrame(results['aggregate_dominance_matrix'], index=alternative_names, columns=alternative_names))

# Step 8: Final Ranking (derived from E)
print("\nStep 8: Final Ranking (interpreting E)")
ranking = solver.get_ranking() # Get ranking calculated by the solver
# Explanation based on the simple net score used in _determine_ranking
print("  (Ranking based on net dominance score: sum(row E) - sum(col E))") 
for i, name in enumerate(ranking):
    print(f"  {i+1}. {name}")


------ DETAILED STEP-BY-STEP RESULTS ------

Step 1: Normalized Decision Matrix (R)


,C1(Harga),C2(Kamera),C3(Memori),C4(Berat),C5(Keunikan)
HP1,0.553,0.550,0.601,0.577,0.599
HP2,0.553,0.629,0.526,0.577,0.599
HP3,0.623,0.550,0.601,0.577,0.532



Step 2: Weighted Normalized Matrix (V)


,C1(Harga),C2(Kamera),C3(Memori),C4(Berat),C5(Keunikan)
HP1,2.767,2.200,1.804,2.309,1.197
HP2,2.767,2.514,1.578,2.309,1.197
HP3,3.113,2.200,1.804,2.309,1.064



Steps 3 & 4: Concordance and Discordance Analysis
Global Max Difference for Discordance Denominator: 0.346

--- Pair (HP1 vs HP2) ---
  Concordance Set C(HP1,HP2): { C1(Harga), C3(Memori), C4(Berat), C5(Keunikan) }
  Concordance Index C_1,2: 14.0
  Discordance Set D(HP1,HP2): { C2(Kamera) }
  Max Difference in D(HP1,HP2): 0.314
  Discordance Index d_1,2: (0.314 / 0.346) = 0.909

--- Pair (HP1 vs HP3) ---
  Concordance Set C(HP1,HP3): { C1(Harga), C2(Kamera), C3(Memori), C4(Berat), C5(Keunikan) }
  Concordance Index C_1,3: 18.0
  Discordance Set D(HP1,HP3): { None }
  Max Difference in D(HP1,HP3): 0.000
  Discordance Index d_1,3: (0.000 / 0.346) = 0.000

--- Pair (HP2 vs HP1) ---
  Concordance Set C(HP2,HP1): { C1(Harga), C2(Kamera), C4(Berat), C5(Keunikan) }
  Concordance Index C_2,1: 15.0
  Discordance Set D(HP2,HP1): { C3(Memori) }
  Max Difference in D(HP2,HP1): 0.225
  Discordance Index d_2,1: (0.225 / 0.346) = 0.652

--- Pair (HP2 vs HP3) ---
  Concordance Set C(HP2,HP3): { C1(Ha

...

  Discordance Dominance Matrix (G): d_kl <= 0.7021


,HP1,HP2,HP3
HP1,0,0,1
HP2,1,0,1
HP3,0,0,0



Step 7: Aggregate Dominance Matrix (E) = F * G


,HP1,HP2,HP3
HP1,0,0,1
HP2,1,0,1
HP3,0,0,0



Step 8: Final Ranking (interpreting E)
  (Ranking based on net dominance score: sum(row E) - sum(col E))
  1. HP2
  2. HP1
  3. HP3
